# Intro to Recurrent Neural Networks: Character-wise RNN Walkthrough

Following along the content in DLND

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we load the text file and convert it into integers for network to use

In [2]:
with open('anna.txt', 'r') as f:
    text = f.read()
vocab = set(text)
vocab2int = {c: i for i, c in enumerate(vocab)}
int2vocab = dict(enumerate(vocab))
chars = np.array([vocab2int[c] for c in text], dtype=np.int32)

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
chars[:100]

array([64,  0, 25, 52, 67, 30, 72, 63, 38, 60, 60, 60, 11, 25, 52, 52, 56,
       63, 68, 25, 69, 22, 29, 22, 30, 23, 63, 25, 72, 30, 63, 25, 29, 29,
       63, 25, 29, 22, 13, 30, 21, 63, 30, 73, 30, 72, 56, 63, 81, 16,  0,
       25, 52, 52, 56, 63, 68, 25, 69, 22, 29, 56, 63, 22, 23, 63, 81, 16,
        0, 25, 52, 52, 56, 63, 22, 16, 63, 22, 67, 23, 63, 79, 82, 16, 60,
       82, 25, 56, 33, 60, 60,  1, 73, 30, 72, 56, 67,  0, 22, 16], dtype=int32)

Now need to split up the date into batches, and into training and validation sets. Should be making a test set here, but the test will be if the network can generate new text.

Make both input and target arrays. The targets are the same as outputs, except shifted one character over. Also drop the last bit of data so that there are only completely full batches.

The idea is to make a 2D matrix where the number of rows is equal to the number of batches. Each row will be one long concatenated string from the character data. Split this data into a training set and validation set using the split_frac keyword. This will keep 90% 

In [5]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ----------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    Returns train_x, train_y, val_x, val_y
    """
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches * slice_size]
    y = chars[1: n_batches * slice_size + 1]
    
    # Split the data into batch-size slices, then stack them into a 2D matrix
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch-size x n_batches * num_steps
    
    # Split into training and validation sets, keep the first split_frac batches for training
    split_idx = int(n_batches * split_frac)
    train_x, train_y = x[:, :split_idx * num_steps], y[:, :split_idx * num_steps]
    val_x, val_y = x[:, split_idx * num_steps:], y[:, split_idx * num_steps:]
    
    return train_x, train_y, val_x, val_y

In [6]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 200)

In [7]:
train_x.shape

(10, 178400)

In [8]:
train_x[:, :10]

array([[64,  0, 25, 52, 67, 30, 72, 63, 38, 60],
       [78, 16, 58, 63,  0, 30, 63, 69, 79, 73],
       [63, 54, 25, 67, 54,  0, 22, 16, 75, 63],
       [79, 67,  0, 30, 72, 63, 82, 79, 81, 29],
       [63, 67,  0, 30, 63, 29, 25, 16, 58, 39],
       [63,  3,  0, 72, 79, 81, 75,  0, 63, 29],
       [67, 63, 67, 79, 60, 58, 79, 33, 60, 60],
       [79, 63,  0, 30, 72, 23, 30, 29, 68, 45],
       [ 0, 25, 67, 63, 22, 23, 63, 67,  0, 30],
       [30, 72, 23, 30, 29, 68, 63, 25, 16, 58]], dtype=int32)

Another function to grab batches out of arrays made by split data. Each batch will be a sliding window on these arrays with size batch_size X num_steps. For example, if we want our network to train on a sequence of 100 characters, num_steps = 100. For the next batch, we'll shift this window the next sequence of num_steps characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [9]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size / num_steps)
    for b in range(n_batches):
        yield [x[:, b * num_steps: (b + 1) * num_steps] for x in arrs]
    

In [10]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
             learning_rate=0.001, grad_clip=5, sampling=False):
    
    if sampling == True:
        batch_size, num_steps = 1, 1
        
    tf.reset_default_graph()
    
    # Declare placeholders we will feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # One-hot encoding the input characters
    x_one_hot = tf.one_hot(inputs, num_classes)
    y_one_hot = tf.one_hot(targets, num_classes)
    
    # Build the RNN layers
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    # Run the data through the RNN layers
    rnn_inputs = [tf.squeeze(i, squeeze_dims=[1]) for i in tf.split(x_one_hot, num_steps, 1)]
    outputs, state = tf.contrib.rnn.static_rnn(cell, rnn_inputs, initial_state=initial_state)
    final_state = state
    
    # Reshape output so its a bunch of rows, one row for each cell output
    seq_output = tf.concat(outputs, axis=1)
    output = tf.reshape(seq_output, [-1, lstm_size])
    
    # Now connect the RNN outputs to a softmax layer and calculate the cost
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(num_classes))
        
    logits = tf.matmul(output, softmax_w) + softmax_b
    preds = tf.nn.softmax(logits, name='predictions')
    
    y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    cost = tf.reduce_mean(loss)
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    # Export the nodes
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state', 
                    'keep_prob', 'cost', 'preds', 'optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

# Hyperparameters 
Defining the hyperparameters for the network. lstm_size and num_layers set the number of hidden units in the LSTM layers and the number of LSTM layers, respectively. Of course, making these bigger will improve the network's performance but have to watch out for overfitting. If validation loss is much larger than the training loss, it is probably overfitting. Decrease the size of the network or decrease the dropout keep probability

In [11]:
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers = 2
learning_rate = 0.001

# Training
Time for training which is pretty straightforward. Pass in some data, and get an LSTM state back.  Then pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by save_every_n) calculate the validation loss and save a checkpoint.

In [12]:
!mkdir -p checkpoints/anna

In [13]:
epochs = 20
save_every_n = 200
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

model = build_rnn(len(vocab),
                 batch_size=batch_size,
                 num_steps=num_steps,
                 learning_rate=learning_rate,
                 lstm_size=lstm_size,
                 num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/anna20.ckpt')
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x,y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e * n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                   model.targets: y,
                   model.keep_prob: 0.5,
                   model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.cost, model.final_state, model.optimizer],
                                               feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{}'.format(e+1, epochs),
                 'Iteration {}/{}'.format(iteration, iterations),
                 'Training loss: {:.4f}'.format(loss/b),
                 '{:.4f} sec / batch'.format((end-start)))
            
            if (iteration % save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x,y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                           model.targets: y,
                           model.keep_prob: 1.,
                           model.initial_state: new_state}
                    batch_loss, new_state = sess.run([model.cost, model.final_state],
                                                    feed_dict=feed)
                    val_loss.append(batch_loss)
                    
                print('Validation loss', np.mean(val_loss), 'Saving checkpoint!')
                saver.save(sess, "checkpoints/anna/i{}_l{}_{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))
                
            

ValueError: Attempt to reuse RNNCell <tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.BasicLSTMCell object at 0x7f6b6c2ccbe0> with a different variable scope than its first use.  First use of cell was with scope 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell', this attempt is with scope 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell'.  Please create a new instance of the cell if you would like it to use a different set of weights.  If before you were using: MultiRNNCell([BasicLSTMCell(...)] * num_layers), change to: MultiRNNCell([BasicLSTMCell(...) for _ in range(num_layers)]).  If before you were using the same cell instance as both the forward and reverse cell of a bidirectional RNN, simply create two instances (one for forward, one for reverse).  In May 2017, we will start transitioning this cell's behavior to use existing stored weights, if any, when it is called with scope=None (which can lead to silent model degradation, so this error will remain until then.)